In [ ]:
import pandas as pd
import scipy as stats
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sb
sb.set_style('whitegrid')


In [47]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_train = "drive/My Drive/Datasets/ibm_data.csv"

In [ ]:
data = pd.read_csv(path_train)
#data.head()

In [ ]:
#data.head()
#data.info()
#data.describe()

In [ ]:
#data.isnull().sum()

In [ ]:
# Bounce rate
data["bounce_rate_%"] = ((data.VISIT - data.ENGD_VISIT)/data.VISIT)*100

In [ ]:
# DATA PREPROCESSING

data.dropna(subset=['city','st','operating_sys','page_vw_ts','Unnamed: 0'], inplace=True)
data.device_type.fillna(value='unkown_device', inplace=True)
data.sec_lvl_domn.fillna(value='unkown_domain', inplace=True)
data.dropna(inplace=True)
data.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
data.drop(labels=['wk', 'mth', 'yr'], axis=1, inplace=True)
data.page_vw_ts = pd.to_datetime(data.page_vw_ts)

In [ ]:
# Filtering Dates to 4-6-2019
data = data[data.page_vw_ts.dt.day == 4  ]
data = data[data.page_vw_ts.dt.year == 2019]
data = data[data.page_vw_ts.dt.month == 6]

In [95]:
# IP Addres's that has total views greater than 24
ip_views = pd.DataFrame(data.groupby('ip_addr').VIEWS.sum().sort_values())
unique_ip_address = ip_views[ip_views.VIEWS > 24].index

# These are the filterd IP's on which we have to find Infomation.
unique_ip_address

Index(['d55be82cb6d9f69a9841ec04ef5ffb9244a6720d6b3aa0c3d94f7735fb07ffbb',
       '16ebc267de6c5c886c7c515fbac4b9137abe0611f8ceba82835faa44913e1ad1',
       'e90e827d62d57af6830ac5663f0d9abbddd622e12e5558a49e437d9efd69bf12',
       '156cc4a2692b21b46e80e6b4105d04f3928fdbf6e401f299fb34354eb7bb837a',
       'ee308ca683ba4d2e050bf99e73c70ef33c9d01a1ca15c1e71de8b42947a35c5a',
       'b0012e61da344633861f8ad3bcf73da78c58af0d9e34420ec58c68ffb93c2ee1',
       '269ea637368eee4fc473267bbe95cba0112652b1ddeef565f60e769f7c488fda',
       'd60a28c306f69915bbe7ed386fbdfecf158390d10b4cb8c8ca5b1065e2260001',
       '05307471f83798ed4957ef8f66159efa7c9ec77e684c727b85a93ea718db7587',
       'f9524573a2f260d2787af49cdcde43ac01aa1092e7f5231b7382577610254128',
       ...
       'f81ff9706f756e3fc48cc25d4c6b944f71be2baffd09074493b6e974495b2f78',
       'e121aaaba8c089195f180bc3d33e30c1ce3085bfb42d5045420003ba667ead14',
       '67db3c9f309f579154aad8076fd46f78e0101148dc5a35a804a4a2a722d8e44c',
       'f3e5ee

In [ ]:
#This loop will find the views for each hour and also find the corresponding bounce rate( i used median for bounce rate because mean was giving wrong results)

ip_hour_avg = pd.DataFrame([range(24)])
count=0
bounce_rate = []
for ip_adres in list(unique_ip_address):
    
    print("Iteration No. {}".format(count))
    a_data = data[data.ip_addr==ip_adres]
    b_rate = a_data['bounce_rate_%'].median()
    bounce_rate.append(b_rate)
    
    hour_list = []
    for i in range(24):
        x = a_data[a_data.page_vw_ts.dt.hour == i].VIEWS.sum()
        hour_list.append(x)
    ip_dataframe = pd.DataFrame([hour_list])    
    ip_hour_avg = pd.concat([ip_hour_avg,ip_dataframe])
    count+=1
    
# This loop will run 7219 times and will take around 10 mins

In [91]:
# Shape of each hour dataset
ip_hour_avg.shape

(7219, 27)

In [93]:
#ip_hour_avg.drop(['hour_avg', 'daily_avg', 'avg_bounce_rate'], axis=1, inplace=True)

# Each hour Dataset views will look like this
ip_hour_avg.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0,0,0,2,1,0,0,2,1,4,1,4,1,2,1,2,0,1,0,2,0,1,0,0
1,0,1,0,0,2,0,2,3,0,0,2,3,0,2,2,0,2,2,0,0,2,1,0,1
2,0,1,0,0,1,1,1,4,1,4,1,0,3,0,2,2,0,0,0,3,1,0,0,0
3,0,0,2,0,0,0,3,3,0,4,0,0,3,2,2,0,4,0,0,0,0,0,2,0
4,1,0,7,0,0,0,0,1,1,5,0,2,0,0,0,3,1,0,0,0,1,0,3,0


In [ ]:
# Some valuable insights from Dataset like hour_avg, daily_avg and corresponding Bounce rate

#ip_hour_avg = ip_hour_avg.iloc[1:,:]
ip_hour_avg = ip_hour_avg.set_index(np.arange(0,ip_hour_avg.shape[0]))
ip_sum = ip_hour_avg.sum(axis=1)
ip_mean = ip_hour_avg.mean(axis=1)
ip_hour_avg['hour_avg'] = ip_mean
ip_hour_avg['daily_avg'] = ip_sum
ip_hour_avg['avg_bounce_rate'] = bounce_rate

In [96]:
# After finging Insights  dataset will look like this
ip_hour_avg.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,hour_avg,daily_avg,avg_bounce_rate
0,0,0,0,2,1,0,0,2,1,4,1,4,1,2,1,2,0,1,0,2,0,1,0,0,1.041667,25,100.0
1,0,1,0,0,2,0,2,3,0,0,2,3,0,2,2,0,2,2,0,0,2,1,0,1,1.041667,25,0.0
2,0,1,0,0,1,1,1,4,1,4,1,0,3,0,2,2,0,0,0,3,1,0,0,0,1.041667,25,100.0
3,0,0,2,0,0,0,3,3,0,4,0,0,3,2,2,0,4,0,0,0,0,0,2,0,1.041667,25,0.0
4,1,0,7,0,0,0,0,1,1,5,0,2,0,0,0,3,1,0,0,0,1,0,3,0,1.041667,25,100.0


In [ ]:
#This loop will find the info about are filterd IP's 

#Sample IP_INFO
ip_info = data.loc[data.ip_addr =='16ebc267de6c5c886c7c515fbac4b9137abe0611f8ceba82835faa44913e1ad1', 'ctry_name':'ip_addr'].head(1)
count=0

for ip_adres in list(unique_ip_address):
    print("Iteration No. {}".format(count))
    a = data.loc[data.ip_addr == ip_adres, 'ctry_name':'ip_addr'].head(1)
    ip_info = pd.concat([ip_info, a])
    count+=1
    
ip_info = ip_info.iloc[1:,:]
ip_info = ip_info.set_index(np.arange(0,ip_info.shape[0]))    
    
# This loop will run 7219 times and take almost 5 mins.

In [97]:
# IP_Info. will look like this 
ip_info.head()

,ctry_name,intgrtd_mngmt_name,intgrtd_operating_team_name,city,st,sec_lvl_domn,device_type,operating_sys,ip_addr
0,Germany,DACH,Europe,BIELEFELD-MITTE,NORDRHEIN-WESTFALEN,unkown_domain,unkown_device,MICROSOFT_WINDOWS10,d55be82cb6d9f69a9841ec04ef5ffb9244a6720d6b3aa0...
1,"Korea, Republic of",Korea,Asia Pacific,SEOUL,SEOUL TEUKBYEOLSI,unkown_domain,unkown_device,MICROSOFT_WINDOWS10,16ebc267de6c5c886c7c515fbac4b9137abe0611f8ceba...
2,Ireland,UKI,Europe,DUBLIN,DUBLIN,unkown_domain,unkown_device,MACINTOSH_OS X 10.13,e90e827d62d57af6830ac5663f0d9abbddd622e12e5558...
3,Egypt,Middle East & Africa,Middle East & Africa,AL QAHIRAH,AL QAHIRAH,unkown_domain,unkown_device,MICROSOFT_WINDOWS10,156cc4a2692b21b46e80e6b4105d04f3928fdbf6e401f2...
4,United States,United States,North America,BOSTON,MASSACHUSETTS,BCBSMA.COM,unkown_device,MICROSOFT_WINDOWS10,ee308ca683ba4d2e050bf99e73c70ef33c9d01a1ca15c1...


In [98]:
# And ip_hour Dataset will look like this 
ip_hour_avg.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,hour_avg,daily_avg,avg_bounce_rate
0,0,0,0,2,1,0,0,2,1,4,1,4,1,2,1,2,0,1,0,2,0,1,0,0,1.041667,25,100.0
1,0,1,0,0,2,0,2,3,0,0,2,3,0,2,2,0,2,2,0,0,2,1,0,1,1.041667,25,0.0
2,0,1,0,0,1,1,1,4,1,4,1,0,3,0,2,2,0,0,0,3,1,0,0,0,1.041667,25,100.0
3,0,0,2,0,0,0,3,3,0,4,0,0,3,2,2,0,4,0,0,0,0,0,2,0,1.041667,25,0.0
4,1,0,7,0,0,0,0,1,1,5,0,2,0,0,0,3,1,0,0,0,1,0,3,0,1.041667,25,100.0


In [ ]:
# Joining Both Datasets (ip_info, ip_hour_avg) 

new_ip_data = pd.merge(ip_info, ip_hour_avg, on=ip_info.index, how='outer')
new_ip_data.drop(['key_0'], axis=1, inplace=True)

In [ ]:
# Changing Columns Name  to strings just for simplicity in future filtering... 

new_ip_data.columns = [                  'ctry_name',          'intgrtd_mngmt_name',
       'intgrtd_operating_team_name',                        'city',
                                'st',                'sec_lvl_domn',
                       'device_type',               'operating_sys',
                           'ip_addr',                             '0_hour',
                                   '1_hour',                             '2_hour',
                                   '3_hour',                             '4_hour',
                                   '5_hour',                             '6_hour',
                                   '7_hour',                             '8_hour',
                                   '9_hour',                            '10_hour',
                                  '11_hour',                            '12_hour',
                                  '13_hour',                            '14_hour',
                                  '15_hour',                            '16_hour',
                                  '17_hour',                            '18_hour',
                                  '19_hour',                            '20_hour',
                                  '21_hour',                            '22_hour',
                                  '23_hour',                    'hour_avg',
                         'daily_avg',             'avg_bounce_rate']

In [99]:
# This is our final Dataset 
new_ip_data.head()

,ctry_name,intgrtd_mngmt_name,intgrtd_operating_team_name,city,st,sec_lvl_domn,device_type,operating_sys,ip_addr,0_hour,1_hour,2_hour,3_hour,4_hour,5_hour,6_hour,7_hour,8_hour,9_hour,10_hour,11_hour,12_hour,13_hour,14_hour,15_hour,16_hour,17_hour,18_hour,19_hour,20_hour,21_hour,22_hour,23_hour,hour_avg,daily_avg,avg_bounce_rate
0,Germany,DACH,Europe,BIELEFELD-MITTE,NORDRHEIN-WESTFALEN,unkown_domain,unkown_device,MICROSOFT_WINDOWS10,d55be82cb6d9f69a9841ec04ef5ffb9244a6720d6b3aa0...,0,0,0,2,1,0,0,2,1,4,1,4,1,2,1,2,0,1,0,2,0,1,0,0,1.041667,25,100.0
1,"Korea, Republic of",Korea,Asia Pacific,SEOUL,SEOUL TEUKBYEOLSI,unkown_domain,unkown_device,MICROSOFT_WINDOWS10,16ebc267de6c5c886c7c515fbac4b9137abe0611f8ceba...,0,1,0,0,2,0,2,3,0,0,2,3,0,2,2,0,2,2,0,0,2,1,0,1,1.041667,25,0.0
2,Ireland,UKI,Europe,DUBLIN,DUBLIN,unkown_domain,unkown_device,MACINTOSH_OS X 10.13,e90e827d62d57af6830ac5663f0d9abbddd622e12e5558...,0,1,0,0,1,1,1,4,1,4,1,0,3,0,2,2,0,0,0,3,1,0,0,0,1.041667,25,100.0
3,Egypt,Middle East & Africa,Middle East & Africa,AL QAHIRAH,AL QAHIRAH,unkown_domain,unkown_device,MICROSOFT_WINDOWS10,156cc4a2692b21b46e80e6b4105d04f3928fdbf6e401f2...,0,0,2,0,0,0,3,3,0,4,0,0,3,2,2,0,4,0,0,0,0,0,2,0,1.041667,25,0.0
4,United States,United States,North America,BOSTON,MASSACHUSETTS,BCBSMA.COM,unkown_device,MICROSOFT_WINDOWS10,ee308ca683ba4d2e050bf99e73c70ef33c9d01a1ca15c1...,1,0,7,0,0,0,0,1,1,5,0,2,0,0,0,3,1,0,0,0,1,0,3,0,1.041667,25,100.0


So finally i have prepared a dataset that we can train our model on but before this i have to make some changes in it like labeling classes, till now i have worked on just three rules and will work on others 



How to consider the second level domain , becoz one ip comes with different sec lvl domains , but in my model 
i have to only consider unique IP , for my model to understand 

Data comparision for taking out the avg is a challenge 

Data preparation is the toughest , as the data is not ready , lot of data cleaning and preprocessing is required 